## Overview
** Input: t0, t1, ..., tN 에 reviewer가 남긴 review ** <br>
t시점에 남긴 review는 문장 단위로 split된 list 형태 (max 문장 개수: 288)
1. 한 문장을 char-level matrix로 변환 (transform lower case)
2. CNN 적용
3. CNN 결과를 summation하여 한 review의 embedding vector로 표현 

In [46]:
import numpy as np
import pickle
import re

import tensorflow as tf
import tensorflow.contrib.slim as slim

In [117]:
max_len_review = 288

In [110]:
with open('./data.pkl', 'rb') as f:
    data = pickle.load(f)

In [111]:
for k, val in data.items():
    test_review = val[0]['reviewText']
    print(k, val)
    break

A14UQBG3HNR785 [{'asin': 'B00002SAO2', 'reviewText': ['At first I thought that this would be a fun game.', 'After a while, it got very tiresome and the parrot was so annoying.', 'I found a few of the puzzles almost impossible to solve, and ended up needing 2 spoilers.', "The graphics are OK, but it doesn't hold a candle to Myst or Riven.", 'Overall, if you have nothing better to do with your money, and have nothing else to play, then go for it.', "But if you want something REALLY good, you haven't found it yet.", 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', 'ㅁ', '

### sample data
**reviewerId**: <br> 
A14UQBG3HNR785 <br> <br>
**first item asin**: <br>
B00002SAO2 <br> <br>
**first item review**: <br>
['At first I thought that this would be a fun game.', <br>
 'After a while, it got very tiresome and the parrot was so annoying.', <br>
 'I found a few of the puzzles almost impossible to solve, and ended up needing 2 spoilers.', <br>
 "The graphics are OK, but it doesn't hold a candle to Myst or Riven.", <br>
 'Overall, if you have nothing better to do with your money, and have nothing else to play, then go for it.', <br>
 "But if you want something REALLY good, you haven't found it yet.", <br>
 "ㅁ" ... ]

In [112]:
test_review

['At first I thought that this would be a fun game.',
 'After a while, it got very tiresome and the parrot was so annoying.',
 'I found a few of the puzzles almost impossible to solve, and ended up needing 2 spoilers.',
 "The graphics are OK, but it doesn't hold a candle to Myst or Riven.",
 'Overall, if you have nothing better to do with your money, and have nothing else to play, then go for it.',
 "But if you want something REALLY good, you haven't found it yet.",
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 'ㅁ',
 

In [36]:
def sent_to_mtx(sentence, row_size=1000):
    """
    sentence to character-level matrix
    """
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}\n"
    sentence = sentence.lower()
    if len(sentence) > row_size:
        sentence = sentence[:row_size]
    char_mtx = np.zeros([row_size, len(alphabet)], dtype=np.float32)
    for i, char in enumerate(sentence):
        if char in alphabet:
            char_mtx[i, alphabet.index(char)] = 1
    return char_mtx

In [103]:
def review_to_mtx(review):
    mtx = []
    for sent in review:
        mtx.append(sent_to_mtx(sent))
    return np.array(mtx)

In [113]:
test_input = review_to_mtx(test_review)

In [114]:
test_input.shape

(288, 1000, 70)

## char-CNN

In [39]:
char_mtx_row = 1000
_len_alphabet = 70

In [118]:
char_mtx = tf.placeholder(dtype=tf.float32, shape=[None, max_len_review, char_mtx_row, _len_alphabet])

In [119]:
def review_CNN(char_mtx, filter_sizes = [2, 3, 7, 11], filter_nums = [100, 100, 100, 100], stride=[1,1,1,1],
               mlp_units=[200,50], reuse=False, is_training=True):
    ## CNN for review aspect, sentiment extraction
    with tf.variable_scope('review-charCNN', reuse=reuse):
        input_ = tf.reshape(char_mtx, shape=[-1, char_mtx_row, _len_alphabet])
        input_ = tf.expand_dims(input_, axis=3)
        with tf.name_scope('conv-filter-1'):
            filter_shape1 = [filter_sizes[0], _len_alphabet, 1, filter_nums[0]]
            W1 = tf.Variable(tf.truncated_normal(shape=filter_shape1), name='filter-1')
            b1 = tf.Variable(tf.random_uniform(shape=[filter_nums[0]]), name='bias-1')
            conv1 = tf.nn.conv2d(input_, W1, strides=stride, padding='VALID', name='conv-1')
            out1 = tf.nn.relu(tf.nn.bias_add(conv1, b1))
            max1 = tf.reduce_max(out1, axis=1) # batch_size, 1, 100
        with tf.name_scope('conv-filter-2'):
            filter_shape2= [filter_sizes[1], _len_alphabet, 1, filter_nums[1]]
            W2 = tf.Variable(tf.truncated_normal(shape=filter_shape2), name='filter-2')
            b2 = tf.Variable(tf.random_uniform(shape=[filter_nums[1]]), name='bias-2')
            conv2 = tf.nn.conv2d(input_, W2, strides=stride, padding='VALID', name='conv-2')
            out2 = tf.nn.relu(tf.nn.bias_add(conv2, b2))
            max2 = tf.reduce_max(out2, axis=1) # batch_size, 1, 100
        with tf.name_scope('conv-filter-3'):
            filter_shape3 = [filter_sizes[2], _len_alphabet, 1, filter_nums[2]]
            W3 = tf.Variable(tf.truncated_normal(shape=filter_shape3), name='filter-3')
            b3 = tf.Variable(tf.random_uniform(shape=[filter_nums[2]]), name='bias-3')
            conv3 = tf.nn.conv2d(input_, W3, strides=stride, padding='VALID', name='conv-3')
            out3 = tf.nn.relu(tf.nn.bias_add(conv3, b3))
            max3 = tf.reduce_max(out3, axis=1) # batch_size, 1, 100
        with tf.name_scope('conv-filter-4'):
            filter_shape4= [filter_sizes[3], _len_alphabet, 1, filter_nums[3]]
            W4 = tf.Variable(tf.truncated_normal(shape=filter_shape4), name='filter-4')
            b4 = tf.Variable(tf.random_uniform(shape=[filter_nums[3]]), name='bias-4')
            conv4 = tf.nn.conv2d(input_, W4, strides=stride, padding='VALID', name='conv-4')
            out4 = tf.nn.relu(tf.nn.bias_add(conv4, b4))
            max4 = tf.reduce_max(out4, axis=1) # batch_size, 1, 100
        ## concat
        max_concat = tf.squeeze(tf.concat([max1, max2, max3, max4], axis=2), axis=1) # batch_size, 400
    ## MLP for feature reduction
    with tf.variable_scope('review-MLP', reuse=reuse):
        fc1 = slim.fully_connected(max_concat, mlp_units[0])
        fc2 = slim.fully_connected(fc1, mlp_units[1])
    result = tf.reshape(fc2, shape=[-1, max_len_review, mlp_units[1]])
    return tf.reduce_sum(result, axis=1)

In [120]:
result = review_CNN(char_mtx, reuse=True)

In [121]:
result

<tf.Tensor 'Sum:0' shape=(?, 50) dtype=float32>